In [14]:
import sys
sys.path.append('../')

import copy
import glob

from typing import Tuple, Union, List
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm as tqdm

import torch
import torch.nn as nn

import torch_geometric as pyg
from torch_geometric.utils.convert import from_networkx

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from IPython.display import clear_output

%matplotlib inline
%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
G = nx.readwrite.read_gpickle('data/network.gpickle')

In [7]:
node_attrs = list(next(iter(G.nodes(data=True)))[-1].keys())
node_attrs = node_attrs[3:]
pyg_graph = from_networkx(G, group_node_attrs=node_attrs)
pyg_graph.to(device)
print(node_attrs)

['labor_force_rate', 'housing_sales_num_Total:', 'housing_sales_num_Total:!!Less than $10,000', 'housing_sales_num_Total:!!$10,000 to $14,999', 'housing_sales_num_Total:!!$15,000 to $19,999', 'housing_sales_num_Total:!!$20,000 to $24,999', 'housing_sales_num_Total:!!$25,000 to $29,999', 'housing_sales_num_Total:!!$30,000 to $34,999', 'housing_sales_num_Total:!!$35,000 to $39,999', 'housing_sales_num_Total:!!$40,000 to $49,999', 'housing_sales_num_Total:!!$50,000 to $59,999', 'housing_sales_num_Total:!!$60,000 to $69,999', 'housing_sales_num_Total:!!$70,000 to $79,999', 'housing_sales_num_Total:!!$80,000 to $89,999', 'housing_sales_num_Total:!!$90,000 to $99,999', 'housing_sales_num_Total:!!$100,000 to $124,999', 'housing_sales_num_Total:!!$125,000 to $149,999', 'housing_sales_num_Total:!!$150,000 to $174,999', 'housing_sales_num_Total:!!$175,000 to $199,999', 'housing_sales_num_Total:!!$200,000 to $249,999', 'housing_sales_num_Total:!!$250,000 to $299,999', 'housing_sales_num_Total:!!$

In [43]:
ds = torch.load('datasets/2020-12-28.dat')

node_to_idx = np.vectorize(dict(zip(pyg_graph.id, range(len(pyg_graph.id)))).get)
X = pyg_graph.x[node_to_idx(ds.X)].cpu().numpy()
y = ds.targets

from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=42)

In [45]:
model = LinearRegression()

In [46]:
model.fit(X_train, y_train)
model.score(X_validation, y_validation)

-0.19586784836589333